# Advent of Code - 2024 - Day 6 - Problem 2

https://adventofcode.com/2024/day/5

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day6.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

## Create Maze Class

In [2]:
class Maze:

    def __init__(self, data):
        self.data = list(map(list, data))

    def is_valid(self, location):
        row, col = location
        if row < 0 or row >= len(self.data): return False
        if col < 0 or col >= len(self.data[row]): return False
        return True

    def get_value(self, location):
        row, col = location
        return self.data[row][col]
    
    def find_value(self, value):
        for row in range(len(self.data)):
            for col in range(len(self.data[row])):
                if self.data[row][col] == value: return (row,col)
        return None
    
    def get_unobstructed_locations(self):
        locations = set()
        for row in range(len(self.data)):
            for col in range(len(self.data[row])):
                if self.data[row][col] == ".": locations.add((row,col))
        return locations

## Create Cursor Class

In [3]:
ORIENTATION_UP = 0
ORIENTATION_RIGHT = 1
ORIENTATION_DOWN = 2
ORIENTATION_LEFT = 3
class Cursor:

    def __init__(self, location, orientation):
        self.location = location
        self.orientation = orientation

    def __str__(self):
        return f"({self.location[0]},{self.location[1]},{self.orientation})"

    # STATIC
    def get_next_location(self, location, orientation):

        row, col = location

        if   orientation == ORIENTATION_UP:    return (row-1, col)
        elif orientation == ORIENTATION_RIGHT: return (row  , col+1)
        elif orientation == ORIENTATION_DOWN:  return (row+1, col)
        else:                                  return (row  , col-1) # LEFT

    def get_next_locations(self):
        location = self.location
        orientation = self.orientation
        for idx in range(4):
            yield Cursor(self.get_next_location(location, orientation), orientation)
            orientation = (orientation + 1) % 4

## Create Guard Class

In [4]:
GUARD_UP = "^"

class MoveFailure(Exception):
    pass

class Guard:

    def __init__(self, maze, cursor, obstructed_location):
        self.maze = maze
        self.cursor = cursor
        self.path = list()
        self.str_path = set()
        self.obstructed_location = obstructed_location
        
        self.path.append(cursor)
        self.str_path.add(str(cursor))

    def is_obstructed(self, location):
        if self.maze.get_value(location) == "#": return True
        if self.obstructed_location != None and self.obstructed_location == location: return True
        return False

    def move(self):

        for next_cursor in self.cursor.get_next_locations():
            
            # See if we've moved off the map.
            #
            if not self.maze.is_valid(next_cursor.location): 
                return True

            # Move if the direction isn't blocked.
            #
            if not self.is_obstructed(next_cursor.location):

                if str(next_cursor) in self.str_path:
                    raise MoveFailure("Loop detected.")

                self.cursor = next_cursor
                self.path.append(self.cursor)
                self.str_path.add(str(self.cursor))

                return False
            
        raise Exception("No possible moves.")
    
    def can_escape(self):
        try:
            while (not self.move()):
                pass
            return True
        except MoveFailure:
            return False        

## Move Guard Around Maze

In [5]:
maze = Maze(DATA)

location = maze.find_value(GUARD_UP)
cursor = Cursor(location, ORIENTATION_UP)

unobstructed_locations = maze.get_unobstructed_locations()

count = 0
idx = 0
for unobstructed_location in unobstructed_locations:
    idx += 1
    guard = Guard(maze, cursor, unobstructed_location)
    if not guard.can_escape():
        count += 1

print(f"count = {count}")

count = 1705
